<a href="https://colab.research.google.com/github/Ch4rlesSm1th99/Congestion-Predictions/blob/main/Congestion_on_a_bridge_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's take a look at whether we can predict the number of bicycle trips across Seattle's Fremont Bridge based on weather, season, and other factors. We have a dataset with number of bikers that crossed the bridge on a particular day and some accompanying data for that day. We will perform a simple linear regression to relate weather and other information to bicycle counts, in order to estimate how a change in any one of these parameters affects the number of riders on a given day.

Let's download the dataset and have a look at it.

In [ ]:
!gdown 12OEFoq_65x6Sy4doc24FwHu1ujKpbXSn

Downloading...
From: https://drive.google.com/uc?id=12OEFoq_65x6Sy4doc24FwHu1ujKpbXSn
To: /content/bikers_data.csv
100% 213k/213k [00:00<00:00, 3.31MB/s]


In [ ]:
import pandas as pd
import numpy as np


Examine the columns in the dataset.

In [ ]:
# Load Data
data = pd.read_csv("bikers_data.csv", index_col="Date")
data

,Number of bikers,Mon,Tue,Wed,Thu,Fri,Sat,Sun,holiday,daylight_hrs,Rainfall (in),Temp (F),dry day
Date,,,,,,,,,,,,,
2012-10-03,14084.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11.277359,0.00,56.0,1
2012-10-04,13900.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11.219142,0.00,56.5,1
2012-10-05,12592.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11.161038,0.00,59.5,1
2012-10-06,8024.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11.103056,0.00,60.5,1
2012-10-07,8568.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.045208,0.00,60.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,4552.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.237706,0.01,41.0,0
2019-12-28,3352.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.245532,0.00,44.0,1
2019-12-29,3692.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.254701,0.01,45.5,0


In [ ]:
# We use the following columns for our training.
data_x = data[['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun',
                'holiday', 'daylight_hrs', 'Rainfall (in)',
                'dry day']].to_numpy()

# Separating out the target values.
data_y = data[["Number of bikers"]].to_numpy()

data_x.shape, data_y.shape


((2646, 11), (2646, 1))

## a) Split the dataset into training and testing sets

Split the dataset into training and testing sets. Keep 80% of the data for training and 20% of the data for testing.

## Functions


In [ ]:
#splits data 80:20 train:test sets
def split_data(data_x ,data_y):
  #randomise indexes
  ind = np.arange(data_x.shape[0])
  np.random.shuffle(ind)

  #split the indexes
  train_ind = ind[:int(np.floor(0.8*data_x.shape[0]))]
  test_ind = ind[int(np.floor(0.8*data_x.shape[0])):]

  #sample those indexes from each set of data
  x_train = data_x[train_ind]
  y_train = data_y[train_ind]

  x_test = data_x[test_ind]
  y_test = data_y[test_ind]

  return x_train, y_train, x_test, y_test

#computes x matrix (in the normal equation) by adding column of ones onto array of original x data
def x_matrix(data):
  x_matrix = np.concatenate((np.ones((data.shape[0], 1)), data), axis = 1)
  return x_matrix

#solve normal eq generally
def find_theta(x_matrix,y_data):
  theta = np.linalg.solve(x_matrix.T @ x_matrix , x_matrix.T @ y_data)
  return theta

#makes predictions of y
def predict_y(x_matrix, theta):
  y_pred = x_matrix @ theta
  return y_pred

#find absolute average error
def abs_error(y_test, y_pred):
  mean_squared_err = ((y_pred - y_test)**2).mean()
  av_error = mean_squared_err**0.5
  return av_error


In [ ]:
#split data 80:20 and assign variables to each piece of data
x_train1, y_train1, x_test1, y_test1 = split_data(data_x , data_y)


## b) Train a linear regression model

Build a linear regression model for predicting the numner of bikers using the mean squared error loss function.

In [ ]:
#compute x matrix for the training data for normal eq
x_train1_m = x_matrix(x_train1)

#solve specifically for training data
theta1 = find_theta(x_train1_m, y_train1)

## c) Predict and Evaluate

Predict on the test set and calculate the average absolute error between predictions and true value.

Comment on the results.

In [ ]:
#compute x matrix for test data in order to make y predictions
x_test1_m = x_matrix(x_test1)

#make predictions of y from test data
y_pred1 = predict_y(x_test1_m, theta1)

#computes average absolute error
print(f"on average our model deviates by an absolute error of {int(abs_error(y_test1 , y_pred1))} from the true data")

on average our model deviates by an absolute error of 2577 from the true data


# Comments on model
The absolute average error is scaled to the data and the predicted data roughly matches the same range of the true data. However even for the largest data point in terms of numbers of bikers there is still on average 20-15% of the max value deviation from the true data which is not necessarily very good and so the model can't accrately predict the number of bikers on a specific day. Another point to mention is that when the model made predictions it soemtimes predicted negative numbers of bikers on some days.  







## d) What is the expected number of bikers on a dry non-holiday Monday with 0 inches of rain and 10.5 hours of daylight?

In [ ]:
#inputting data as an array
x_sample_data = np.array([ 1, 1, 0, 0, 0, 0, 0, 0, 0, 10.5, 0, 1])

#transpose and multiply to make predictions
y_sample_prediction = x_sample_data.T @ theta1

print(f"model predicts that there will be {int(y_sample_prediction[0])} bikers on this day")

model predicts that there will be 13342 bikers on this day


## e) Notice that the dataset had an another column 'Temp (F)' but we aren't using it. Let's use that too and do this again.

Add the 'Temp (F)' column to our X data.

In [ ]:
# We use the following columns for our training.
data_x = data[['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun',
                'holiday', 'daylight_hrs', 'Rainfall (in)',
                'dry day', 'Temp (F)']].to_numpy()

# Separating out the target values.
data_y = data[["Number of bikers"]].to_numpy()

data_x.shape, data_y.shape

((2646, 12), (2646, 1))

## f) Split the dataset into training and testing sets again


In [ ]:
#split data 80:20 and assign variables to each piece of data
x_train2, y_train2, x_test2, y_test2 = split_data(data_x , data_y)

## g) Train a linear regression model again

Build a linear regression model for predicting the numner of bikers using the mean squared error loss function.

In [ ]:
#compute x matrix for the training data for normal eq
x_train2_m = x_matrix(x_train2)

#solve normal equation specifically for second set of training data
theta2 = find_theta(x_train2_m, y_train2)


## h) Predict and Evaluate Again!

Predict on the test set and calculate the average absolute error between predictions and true value.

Comment on the results. Did it change? Can you think of any information we can use to make a better(more informed) model?

In [ ]:
#compute x matrix for test data to make y pred
x_test2_m = x_matrix(x_test2)

#make predictions from test data
y_pred2 = predict_y(x_test2_m, theta2)

#calculating average absolute error
print(f"on average our model deviates by an absolute error of {int(abs_error(y_test2 , y_pred2))} from the true data")

on average our model deviates by an absolute error of 2278 from the true data


# Comments on new model
New model predicts a lower absolute average arror than the previous model (15%-10% of max value now)however the error is still not necessarily acceptable to be able to accurately predict the number of bikers on a given day although it has improved upon the previous model and can still provide an indication of the number of bikers on a day. The model still predicts on certain days there will be a negative number of bikers on the bridge. Some other information that might be useful for improving this model is the traffic congestion ie number of vehicles on the brdge on a given day.